(a) Algorithm implementation:
Implement the above hybrid algorithm.

In [42]:
import matplotlib.pyplot as plt
import time
import numpy as np

comparison_counter = 0

# Insertion Sort Algorithm
def insertion_sort(arr, left, right):
    global comparison_counter
    for i in range(left + 1, right + 1):
        temp = arr[i]
        j = i - 1
        while j >= left and arr[j] > temp:
            comparison_counter += 1
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = temp

# Merge Function
def merge(arr, left, mid, right):
    global comparison_counter
    left_arr = arr[left : mid + 1]
    right_arr = arr[mid + 1 : right + 1]

    i, j, k = 0, 0, left 

    l1 = len(left_arr)
    l2 = len(right_arr)
    
    while i < l1 and j < l2:
        comparison_counter += 1
        if left_arr[i] <= right_arr[j]:
            arr[k] = left_arr[i]
            i += 1
        else:
            arr[k] = right_arr[j]
            j += 1
        k += 1

    while i < l1:
        arr[k] = left_arr[i]
        i += 1
        k += 1

    while j < l2:
        arr[k] = right_arr[j]
        j += 1
        k += 1

# Merge Sort Algorithm
def merge_sort(arr, left, right):
    global comparison_counter
    if (left >= right):
        return
    mid = (left + right) // 2
    merge_sort(arr, left, mid)
    merge_sort(arr, mid + 1, right)
    merge (arr, left, mid, right)

# Hybrid Sort Algorithm
def hybrid_sort(arr, left, right, S):
    global comparison_counter
    if right - left <= S:
        insertion_sort(arr, left, right)
    else:
        mid = (left + right) // 2
        hybrid_sort(arr, left, mid, S)
        hybrid_sort(arr, mid + 1, right, S)
        merge(arr, left, mid, right)


(b) Generate input data:
Generate arrays of increasing sizes, in a range from 1,000 to 10 million. For each of the sizes, generate a random dataset of integers in the range of [1, ..., x], where x is the largest number you allow for your datasets.

In [43]:
# Create function to generate dictionary with key being size and value being the list of random integers
def generate_data(start = 1000, end = 10000000, step = 10, seed = 0):
    np.random.seed(seed) # fix the seed for consistency
    dataset = {}
    while start <= end:
        dataset[start] = np.random.randint(1, start + 1, start)
        start *= step
    return dataset

dataset = generate_data()

(c) Analyze time complexity:
Run your program of the hybrid algorithm on the datasets generated in Step (b). Record the number of key comparisons performed in each case.

With the value of S fixed, plot the number of key comparisons over different sizes of the input list n. Compare your empirical results with your theoretical analysis of the time complexity.

In [ ]:
# Dictionary to store results for plotting
comparison_results = {}

# Run the hybrid sort algo multiple times, storing the number of comparisons in an array
for size, arr in dataset.items():
    arr_copy = arr.copy()  

    hybrid_sort(arr_copy, 0, len(arr_copy) - 1, S=10)  

    comparison_results[size] = comparison_counter

    print(f"Size: {size}, Number of comparisons: {comparison_counter}")


Size: 1000, Number of comparisons: 5318
Size: 10000, Number of comparisons: 78602
Size: 100000, Number of comparisons: 926417


In [ ]:
sizes = list(comparison_results.keys())
comparison_counts = list(comparison_results.values())

# Plot Dataset Size vs Number of Comparisons
plt.figure(figsize=(8, 5))
plt.plot(sizes, comparison_counts, linestyle='-', color='b', label="Number of Comparisons")
plt.xlabel("Dataset Size")
plt.ylabel("Number of Comparisons")
plt.title("Hybrid Sort: Dataset Size vs Number of Comparisons")
plt.legend()
plt.show()


With the input size n fixed, plot the number of key comparisons over different values of S. Compare your empirical results with your theoretical analysis of the time complexity.

In [ ]:
# Use fixed dataset and vary S, extracting the sort time and comparison count 
results = {}

for S in range (2, 175):
    # Reset comparison_counter
    comparison_counter = 0
    copied_data = dataset[10000].copy()
    start_time = time.time()
    hybrid_sort(copied_data, 0, len(copied_data) - 1, S)
    end_time = time.time()
    sort_time = end_time - start_time
    results[S] = [sort_time, comparison_counter]

# Extract S values, sort times, and comparison counts from results dictionary
S_values = list(results.keys())
sort_times = [results[S][0] for S in S_values]  # Extract sort times
comparison_counts = [results[S][1] for S in S_values]  # Extract comparison counts

# Plotting S vs Sort Time
plt.figure(figsize=(8, 5))
plt.plot(S_values, sort_times, linestyle='-', color='b', label="Sort Time")
plt.xlabel("S")
plt.ylabel("Sort Time (s)")
plt.title("Hybrid Sort: S vs Sort Time")
plt.legend()
plt.show()

# Plotting S vs Comparison Count
plt.figure(figsize=(8, 5))
plt.plot(S_values, comparison_counts, linestyle='-', color='r', label="Comparison Count")
plt.xlabel("S")
plt.ylabel("Comparison Count")
plt.title("Hybrid Sort: S vs Comparison Count")
plt.legend()
plt.show()

Using different sizes of input datasets, study how to determine an optimal value of S for the best performance of this hybrid algorithm.

In [ ]:
# Function to measure sorting performance for different S values
def measure_sort_performance(num_trials=100):
    sizes = list(range(1, 26))  # Test S values from 1 to 25
    merge_comparisons = []
    insertion_comparisons = []
    
    merge_times = []
    insertion_times = []
    
    for n in sizes:
        merge_comparisons_total = 0
        insertion_comparisons_total = 0
        merge_time_total = 0
        insertion_time_total = 0

        for _ in range(num_trials):  # Run multiple trials for averaging
            arr = np.random.randint(1, n + 1, n)  # Generate small random array

            # Measure Merge Sort comparisons and runtime
            arr_copy = arr.copy()
            global comparison_counter
            comparison_counter = 0
            start_time = time.time()
            merge_sort(arr_copy, 0, len(arr_copy) - 1)
            merge_time_total += time.time() - start_time
            merge_comparisons_total += comparison_counter

            # Measure Insertion Sort comparisons and runtime
            arr_copy = arr.copy()
            comparison_counter = 0
            start_time = time.time()
            insertion_sort(arr_copy, 0, len(arr_copy) - 1)
            insertion_time_total += time.time() - start_time
            insertion_comparisons_total += comparison_counter

        # Compute average key comparisons and runtime
        merge_comparisons.append(merge_comparisons_total / num_trials)
        insertion_comparisons.append(insertion_comparisons_total / num_trials)
        merge_times.append(merge_time_total / num_trials)
        insertion_times.append(insertion_time_total / num_trials)

    # Plot Key Comparisons (Merge Sort vs Insertion Sort)
    plt.figure(figsize=(10, 5))
    plt.plot(sizes, merge_comparisons, label="Merge Sort (Comparisons)", marker="o", linestyle='--', color='blue')
    plt.plot(sizes, insertion_comparisons, label="Insertion Sort (Comparisons)", marker="s", linestyle='-.', color='red')
    plt.xlabel("Array Size (S)")
    plt.ylabel("Average Key Comparisons")
    plt.title("Merge Sort vs Insertion Sort - Key Comparisons")
    plt.legend()
    plt.grid(True)
    plt.show()

    # Plot Runtime (Merge Sort vs Insertion Sort)
    plt.figure(figsize=(10, 5))
    plt.plot(sizes, merge_times, label="Merge Sort (Time)", marker="o", linestyle='--', color='blue')
    plt.plot(sizes, insertion_times, label="Insertion Sort (Time)", marker="s", linestyle='-.', color='red')
    plt.xlabel("Array Size (S)")
    plt.ylabel("Average Runtime (s)")
    plt.title("Merge Sort vs Insertion Sort - Runtime")
    plt.legend()
    plt.grid(True)
    plt.show()

# Run performance measurement
measure_sort_performance(num_trials=100)  # Run 100 trials per size

(d) Compare with original Mergesort:
Implement the original version of Mergesort (as learnt in lecture). Compare its performance against the above hybrid algorithm in terms of the number of key comparisons and CPU times on the dataset with 10 million integers. You can use the optimal value of S obtained in (c) for this task.



In [41]:
# Standard Mergesort Algorithm
optimal_s = 9 # need to replace this with the optimal s

copied_array = dataset[10000000].copy()
# Hybrid Sort Execution
start_time = time.time()
hybrid_sort(arr_copy, 0, len(arr_copy) - 1, optimal_s) 
end_time = time.time()
hybrid_time = end_time - start_time
hybrid_comparisons = comparison_counter

# Standard Mergesort Execution
comparison_counter = 0
copied_array = dataset[10000000].copy()
start_time = time.time()
merge_sort(arr_copy, 0, len(arr_copy) - 1)
end_time = time.time()
merge_time = end_time - start_time
merge_comparisons = comparison_counter


print(f"Hybrid sort takes {hybrid_time}s of runtime and makes {hybrid_comparisons} comparisons to run")
print(f"Merge sort takes {merge_time}s of runtime and makes {merge_comparisons} comparisons to run ")

Hybrid sort takes 25.537683963775635s of runtime and makes 100322625 comparisons to run
Merge sort takes 31.73897695541382s of runtime and makes 118788160 comparisons to run 
